In [1]:
import pandas as pd

import bokeh.io
import bokeh.models
import bokeh.plotting
import bokeh.palettes

import server.model.connection as smc
import server.model.event as sme
import server.season.s2019.view.sixteam

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
conn = smc.pool.getconn()

In [3]:
sme.EventDal.set_current_event('test_data', '2019')
sme.EventDal.set_next_match('030-q')

In [4]:
team = '3223'
num_matches = 3

In [5]:
sql = '''
    SELECT match, task, successes, attempts, last_match
        FROM vw_measures
        WHERE team = %s;
'''
raw_data = pd.read_sql(sql, conn, params=[team])

In [6]:
raw_data.head()

,match,task,successes,attempts,last_match
0,004-q,pickupLoadingCargo,2,2,5
1,004-q,pickSide,0,0,5
2,004-q,crossHabLine,1,1,5
3,004-q,rocketHatch,1,1,5
4,004-q,rocketHatch3,1,1,5


In [7]:
df_all = raw_data.copy(deep=True)
df_all['num_matches'] = df_all.last_match.max()

In [8]:
df_all.tail()

,match,task,successes,attempts,last_match,num_matches
73,027-q,getHatch,1,1,1,5
74,027-q,dropHatchs,0,0,1,5
75,027-q,climb,0,0,1,5
76,027-q,groundCargoPickup,2,2,1,5
77,027-q,rocketCargo1,2,2,1,5


In [13]:
matches_test = list(df_all.match.unique())
matches_test

['004-q', '010-q', '017-q', '023-q', '027-q']

In [10]:
df_recent = raw_data[raw_data.last_match <= num_matches].copy(True)
df_recent['num_matches'] = df_recent.last_match.max()

In [11]:
df_recent.tail()

,match,task,successes,attempts,last_match,num_matches
73,027-q,getHatch,1,1,1,3
74,027-q,dropHatchs,0,0,1,3
75,027-q,climb,0,0,1,3
76,027-q,groundCargoPickup,2,2,1,3
77,027-q,rocketCargo1,2,2,1,3


In [23]:
hatch_tasks = ['getHatch', 'csHatch', 'rocketHatch1', 'rocketHatch2',
               'rocketHatch3']
ha = df_all[df_all.task.isin(hatch_tasks)]
ha

,match,task,successes,attempts,last_match,num_matches
4,004-q,rocketHatch3,1,1,5,5
9,004-q,getHatch,1,1,5,5
13,004-q,rocketHatch1,1,1,5,5
30,010-q,rocketHatch1,1,1,4,5
42,023-q,csHatch,10,10,2,5
55,023-q,getHatch,2,2,2,5
65,027-q,rocketHatch2,2,2,1,5
68,027-q,rocketHatch1,1,1,1,5
70,027-q,rocketHatch3,2,2,1,5
73,027-q,getHatch,1,1,1,5


In [25]:
ha_slim = hatches_all[['match', 'task', 'successes',
                                'num_matches']]
ha_slim['avg_successes'] = ha_slim.successes / ha_slim.num_matches
ha_slim

,match,task,successes,num_matches,avg_successes
4,004-q,rocketHatch3,1,5,0.2
9,004-q,getHatch,1,5,0.2
13,004-q,rocketHatch1,1,5,0.2
30,010-q,rocketHatch1,1,5,0.2
42,023-q,csHatch,10,5,2.0
55,023-q,getHatch,2,5,0.4
65,027-q,rocketHatch2,2,5,0.4
68,027-q,rocketHatch1,1,5,0.2
70,027-q,rocketHatch3,2,5,0.4
73,027-q,getHatch,1,5,0.2


In [28]:
ha = ha_slim.pivot(index='match', columns='task', values='successes')
ha.fillna(0, inplace=True)
ha

task,csHatch,getHatch,rocketHatch1,rocketHatch2,rocketHatch3
match,,,,,
004-q,0.0,1.0,1.0,0.0,1.0
010-q,0.0,0.0,1.0,0.0,0.0
023-q,10.0,2.0,0.0,0.0,0.0
027-q,0.0,1.0,1.0,2.0,2.0


In [30]:
matches = ha.index.values
matches

array(['004-q', '010-q', '023-q', '027-q'], dtype=object)

In [35]:
ha_cds = bokeh.models.ColumnDataSource(ha)
ha_cds.column_names

['match',
 'csHatch',
 'getHatch',
 'rocketHatch1',
 'rocketHatch2',
 'rocketHatch3']

In [44]:
hatch_tasks

['getHatch', 'csHatch', 'rocketHatch1', 'rocketHatch2', 'rocketHatch3']

In [57]:
t1_plot = bokeh.plotting.figure(x_range=matches, title = '1T',
                               plot_height=350, plot_width=500)

In [58]:
t1_plot.vbar_stack(hatch_tasks, x='match', width=0.2,
                   source=ha_cds,
                   color=bokeh.palettes.BuPu5,
                  legend=[' ' + x for x in hatch_tasks])
bokeh.io.show(t1_plot)

In [52]:
ha_cds.column_names

['match',
 'csHatch',
 'getHatch',
 'rocketHatch1',
 'rocketHatch2',
 'rocketHatch3']